In [3]:
import sys
import os
import yaml
import random
from aip import AipSpeech
root_dir = "/xiaobai/"
sys.path.append(root_dir)
import snowboydecoder
class XiaoBai:
    #初始化函数，设置
    def __init__(self,keyword_model):
        self.detector = snowboydecoder.HotwordDetector(keyword_model, sensitivity=0.5)
        self.skills = []
        self.greetings = ["嗯哼.mp3","我在.mp3","请说.mp3"]
        with open(root_dir+"config.yaml") as f:
            config = yaml.load(f)['baidu_yuyin']
            self.client = AipSpeech(config['app_id'], config['api_key'], config['secret_key'])
    #检测到关键字后的操作
    def callback(self):
            self.detector.terminate()
            n = random.randint(0,len(self.greetings)-1)
            notify_sound = root_dir+'resources/greetings/'+self.greetings[n]
            os.system("mpg123 "+notify_sound)            
            res = self.listen_and_recognize()
            if res == "":
                print('你：""(你什么也没说)')
                self.speak("")
            else:
                print("你："+res)
                handled = False
                for skill in self.skills:
                    if skill.handle(res,callback=self.speak):
                        handled = True
                        break
                if not handled:
                    self.speak("小白暂时不会处理呢")
            self.detector.start(detected_callback=self.callback,sleep_time=0.03)
    #添加技能
    def add_skill(self,skill):
        if skill.type == "skill":
            self.skills.append(skill)
    def listen_for_keyword(self):
        try:
            print('Listening...')
            self.detector.start(detected_callback=self.callback,sleep_time=0.03)
        except KeyboardInterrupt:
            print('stop')
        finally:
            self.detector.terminate()            
    #录音和识别函数,调用arecord录音
    def listen_and_recognize(self,length = 4):
        os.system("arecord -d %d -r 16000 -c 1 -t wav -f S16_LE record.wav" % (length,) )    
        with open("./record.wav", 'rb') as fp:
            res = self.client.asr(fp.read(), 'wav', 16000, { 'dev_pid': 1536,})
            if res['err_no']==0:
                return res["result"][0]
            else:
                #print(res)
                return ""
    #调用百度语音合成API进行回复
    def speak(self,text = '你好百度',lang = 'zh',type = 1 , vol = 5, spd = 5 , pit = 5):
        if text == "":
            print('小白：......')
            return
        result  = self.client.synthesis(text, lang, type, {'vol': vol,'spd':spd,'pit':pit})
        # 识别正确返回语音二进制 错误则返回dict
        if not isinstance(result, dict):
            with open('speak.mp3', 'wb') as f:
                f.write(result)
            print('小白：'+text)
            os.system('mpg123 speak.mp3')
        else:
            print('emmmm，小白出错了呢')
            print(result)
            
import abc #利用abc模块实现抽象类
#编写扩展技能的基本格式，has_intent函数检测是否有需要该技能处理的意图，action函数执行对应的处理
class BaseSkill(metaclass=abc.ABCMeta):
    type='skill'
    #参数说明 
    #    text：语音识别的到的文本
    #    callback：反馈文本的处理函数，默认直接打印，也可以传入语音合成函数进行语音回复
    #定义抽象方法，检测是否有需要该技能处理的意图
    @abc.abstractmethod 
    def has_intent(self,text=""):
        pass
    #定义抽象方法，根据意图处理处理信息
    @abc.abstractmethod
    def action(self,text=""):
        pass
    #处理函数，根据意图处理处理信息，返回是否继续检测意图
    def handle(self,text="",callback=print):
        if self.has_intent(text=text):
            self.action(text=text,callback=callback)
            return True
        else:
            return False   


In [2]:
import requests
import json
import yaml

class TalkSkill(BaseSkill):
    def __init__(self):
        root_dir = "/xiaobai/"
        with open(root_dir+"config.yaml") as f:
            config = yaml.load(f)['tuling']
            self.key = config['key']
            self.url = 'http://www.tuling123.com/openapi/api'
    #继承BaseSkill类，必须定义has_intent和action方法
    def has_intent(self,text=""):
        if text!= "":
            return True
        return False
    def action(self,text="",callback=print):
        try:
            req = {'key':self.key,'info':text}
            res = requests.post(url = self.url, data = req)
            #print(res.text)
            jd = json.loads(res.text)
            callback(jd['text'])
        except:
            callback("出错了呢，可能网络不太好")
if __name__ == '__main__':
    s = TalkSkill()
    s.handle("你好呀")
    s.handle("北京明天天气怎么样")
    s.handle("讲个冷笑话")

我最喜欢和有礼貌的孩子聊天了！
北京:周日,晴 西南风3-4级,最低气温13度，最高气温26度
协议某夫是个“妻管严”，太太与他有一个协议：第一条：老婆永远是对的！第二条：如果老婆错了，请参考第一条！！


In [ ]:
keyword_model = root_dir+'resources/小白.pmdl'
xiaobai = XiaoBai(keyword_model=keyword_model)
xiaobai.add_skill(TalkSkill())
xiaobai.listen_for_keyword()

Listening...


INFO:snowboy:Keyword 1 detected at time: 2019-05-11 14:06:08


你：今天天气怎么样北京的
小白：云浮:周六 05月11日,多云 无持续风向微风,最低气温22度，最高气温31度
